<H1>FBN1: Marfan syndrome (Tiecke, 2001)</H1>
<p>Extract phenopackets from the clinical data in <a href="https://pubmed.ncbi.nlm.nih.gov/11175294/" target="__blank">Tiecke et al (2001)</a>.</p>

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.63


In [2]:
PMID = "PMID:11175294"
title = "Classic, atypically severe and neonatal Marfan syndrome: twelve mutations and genotype-phenotype correlations in FBN1 exons 24-40"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_table("input/tiecke_2001.tsv")

In [4]:
df

,Feature/finding,B7,B15,B22,B27,B35,B47,D1,D13,D27,D36,D54,D60
0,AgeLastExamination(years),13,16,32,35,10,3,50,28,35,11,33,8
1,Inheritance,fam.,spor.,fam.,NaN,fam.,fam.,fam.,fam.,fam.,fam.,spor.,spor.
2,Gender,f,f,m,f,f,f,m,f,m,f,m,f
3,HeightOver98P,+,–,+,–,+,+,+,–,+,+,+,+
4,PectusDeformity,car.,exc.,car.,asym.,car.,–,–,–,car.,–,exc.,–
5,WristAndThumbSign,+,+,+,+,+,±,–,NaN,NaN,+,+,+
6,Scoliosis,–,+,+,+,+,–,–,+,–,–,+,+
7,HyperextensibleJoints,–,–,–,–,+,+,+,NaN,NaN,+,–,+
8,Other,HP,HP;D,HP;D,DE,HP;D,HP,HP;D,–,HP;D,–,St,HP;D;En
9,EctopiaLentis,–,+,+,–,–,NaN,+,+,+,+,+,+


In [5]:
# convert to row-based format
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['patient_id'] = dft.index
dft.head()

Feature/finding,AgeLastExamination(years),Inheritance,Gender,HeightOver98P,PectusDeformity,WristAndThumbSign,Scoliosis,HyperextensibleJoints,Other,EctopiaLentis,Myopia(diopters),DilationAscendingAorta(mm),AorticRootReplacement(age),MitralValve,OtherCV,UnusualFeatures,patient_id
B7,13,fam.,f,+,car.,+,–,–,HP,–,+(–2),+(38),–,MVP,–,–,B7
B15,16,spor.,f,–,exc.,+,+,–,HP;D,+,+,+(75),16,MR,AR,+,B15
B22,32,fam.,m,+,car.,+,+,–,HP;D,+,+(–10),+(50),32,MVP,–,–,B22
B27,35,NaN,f,–,asym.,+,+,–,DE,–,+(–2),+(40),34,MVP,AR,+,B27
B35,10,fam.,f,+,car.,+,+,+,HP;D,–,mild,+(28),28,MVP,–,–,B35


In [6]:
# Add in the "unusual features" not in the table from the text
dft.at['B15', 'UnusualFeatures'] = "Thin skin; Decreased body weight"
dft.at['B27', 'UnusualFeatures'] = "Ulnar deviation of finger; Narrow face; Dental malocclusion; Dural ectasia; Patent foramen ovale "

In [7]:
column_mapper_list = list()

In [8]:
items = {
      'HeightOver98P': ['Tall stature', 'HP:0000098'],
      'WristAndThumbSign': ['Arachnodactyly', 'HP:0001166'],
      'Scoliosis': ['Scoliosis', 'HP:0002650'],
      'HyperextensibleJoints': ['Joint hypermobility','HP:0001382'],
      'EctopiaLentis': ["Ectopia lentis", "HP:0001083"],  
}
item_column_mapper_d = hpo_cr.initialize_simple_column_maps(column_name_to_hpo_label_map=items, 
                                                            observed=['+','±'],
                                                            excluded='-')

# Transfer to column_mapper_d
for k, v in item_column_mapper_d.items():
    column_mapper_list.append(v)

In [9]:
option_d = {
    "car.": "Pectus carinatum", 
    "exc.": "Pectus excavatum",
}
pectusMapper = OptionColumnMapper(column_name="PectusDeformity",concept_recognizer=hpo_cr ,option_d=option_d)
column_mapper_list.append(pectusMapper)
pectusMapper.preview_column(dft)

,mapping,count
0,Pectus carinatum (HP:0000768) (observed),4
1,Pectus excavatum (HP:0000767) (observed),2


In [10]:
# Other
other_d = {
    "HP": "High palate",
    "D": "Dolichocephaly", 
    "En": "Deeply set eye", # i.e., Enophthalmus
    "DE": "Dural ectasia", 
    "St": "Striae distensae",
}
otherMapper = OptionColumnMapper(column_name="Other",concept_recognizer=hpo_cr, option_d=other_d)
column_mapper_list.append(otherMapper)
otherMapper.preview_column(dft)

,mapping,count
0,High palate (HP:0000218) (observed),8
1,Dolichocephaly (HP:0000268) (observed),6
2,Dural ectasia (HP:0100775) (observed),1
3,Striae distensae (HP:0001065) (observed),1
4,Deeply set eye (HP:0000490) (observed),1


In [11]:
myopia_d ={
    "+(–2)" : "Mild myopia",
    "+": "Myopia",
    "+(–10)": "High myopia",
    "mild": "Mild myopia",
    "+(–13)": "High myopia",
    "+(–6)": "High myopia"
}
myopiaMapper = OptionColumnMapper(column_name="Myopia(diopters)",concept_recognizer=hpo_cr, option_d=myopia_d)
myopiaMapper.preview_column(dft)
column_mapper_list.append(myopiaMapper)

In [12]:
# DilationAscendingAorta(mm) -- all patients have this feature
aoDilMapper = ConstantColumnMapper(column_name="DilationAscendingAorta(mm)",hpo_id="HP:0002616", hpo_label="Aortic root aneurysm")
column_mapper_list.append(aoDilMapper)
#aoDilMapper.preview_column(dft)

In [13]:
mitral_d = {
    "MVP": "Mitral valve prolapse",
    "MR": "Mitral regurgitation",
    "MVrep": "Mitral regurgitation" # replacement because of severe MR 
}
mitralMapper = OptionColumnMapper(column_name="MitralValve",
                                  concept_recognizer=hpo_cr, option_d=mitral_d)
column_mapper_list.append(mitralMapper)
mitralMapper.preview_column(dft)

,mapping,count
0,Mitral valve prolapse (HP:0001634) (observed),6
1,Mitral regurgitation (HP:0001653) (observed),4


In [14]:
#OtherCV
cv_d = {
    "AR":"Aortic regurgitation",
    "Pneu": "Spontaneous pneumothorax"
}
othercvMapper = OptionColumnMapper(column_name="OtherCV", concept_recognizer=hpo_cr, option_d=cv_d)
column_mapper_list.append(othercvMapper)
othercvMapper.preview_column(dft)

,mapping,count
0,Aortic regurgitation (HP:0001659) (observed),2
1,Spontaneous pneumothorax (HP:0002108) (observed),1


In [15]:
# Variants -- modernized from the original publication to use HGVS
variant_map = {
    "B15":  "c.3037G>A",
    "B47": "c.3058A>G",
    "D54": "c.3082+1G>T", # corresponds to IVS24 +1 G to T, Skipping of exon 24 
    "D60":  "c.3037G>A",
    "B27": "c.3344A>G",
    "B35": "c.3713A>G",
    "D1":  "c.3898A>G",
    "B7": "c.3973G>T",
    "D36": "c.4217A>G",
    "D13": "c.4223G>T",
    "D27": "c.4615C>T",
    "B22": "c.4955G>A"
}

dft["Variants"] = variant_map
fbn1_transcript='NM_000138.5'

vvalidator = VariantValidator(genome_build="hg38", transcript=fbn1_transcript)
variant_d = {}
for v in variant_map.values():
    var = vvalidator.encode_hgvs(v)
    variant_d[v] = var


varMapper = VariantColumnMapper(variant_d=variant_d,
                                variant_column_name='Variants', 
                                default_genotype='heterozygous')

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3037G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3058A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3082+1G>T/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3037G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3344A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3713A>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.3898A>G/NM_000138.5?content-type=application%2Fjson
https://res

In [17]:
sexMapper = SexColumnMapper(male_symbol="m", female_symbol="f", column_name="Gender")
ageMapper = AgeColumnMapper.by_year("AgeLastExamination(years)")
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="patient_id", 
                        age_at_last_encounter_mapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
omim_id = "OMIM:154700"
omim_label = "Marfan syndrome"
mfs = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=mfs)

In [18]:
individuals = encoder.get_individuals()

In [19]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
INFORMATION,NOT_MEASURED,22


In [20]:
individuals = cvalidator.get_error_free_individual_list()
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
B7 (FEMALE; P13Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.3973G>T (heterozygous),Tall stature (HP:0000098); Arachnodactyly (HP:0001166); Pectus carinatum (HP:0000768); High palate (HP:0000218); Mild myopia (HP:0025573); Aortic root aneurysm (HP:0002616); Mitral valve prolapse (HP:0001634)
B15 (FEMALE; P16Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.3037G>A (heterozygous),Arachnodactyly (HP:0001166); Scoliosis (HP:0002650); Ectopia lentis (HP:0001083); Pectus excavatum (HP:0000767); High palate (HP:0000218); Dolichocephaly (HP:0000268); Myopia (HP:0000545); Aortic root aneurysm (HP:0002616); Mitral regurgitation (HP:0001653); Aortic regurgitation (HP:0001659)
B22 (MALE; P32Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.4955G>A (heterozygous),Tall stature (HP:0000098); Arachnodactyly (HP:0001166); Scoliosis (HP:0002650); Ectopia lentis (HP:0001083); Pectus carinatum (HP:0000768); High palate (HP:0000218); Dolichocephaly (HP:0000268); High myopia (HP:0011003); Aortic root aneurysm (HP:0002616); Mitral valve prolapse (HP:0001634)
B27 (FEMALE; P35Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.3344A>G (heterozygous),Arachnodactyly (HP:0001166); Scoliosis (HP:0002650); Dural ectasia (HP:0100775); Mild myopia (HP:0025573); Aortic root aneurysm (HP:0002616); Mitral valve prolapse (HP:0001634); Aortic regurgitation (HP:0001659)
B35 (FEMALE; P10Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.3713A>G (heterozygous),Tall stature (HP:0000098); Arachnodactyly (HP:0001166); Scoliosis (HP:0002650); Joint hypermobility (HP:0001382); Pectus carinatum (HP:0000768); High palate (HP:0000218); Dolichocephaly (HP:0000268); Mild myopia (HP:0025573); Aortic root aneurysm (HP:0002616); Mitral valve prolapse (HP:0001634)
B47 (FEMALE; P3Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.3058A>G (heterozygous),Tall stature (HP:0000098); Arachnodactyly (HP:0001166); Joint hypermobility (HP:0001382); High palate (HP:0000218); Aortic root aneurysm (HP:0002616)
D1 (MALE; P50Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.3898A>G (heterozygous),Tall stature (HP:0000098); Joint hypermobility (HP:0001382); Ectopia lentis (HP:0001083); High palate (HP:0000218); Dolichocephaly (HP:0000268); High myopia (HP:0011003); Aortic root aneurysm (HP:0002616); Mitral regurgitation (HP:0001653)
D13 (FEMALE; P28Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.4223G>T (heterozygous),Scoliosis (HP:0002650); Ectopia lentis (HP:0001083); High myopia (HP:0011003); Aortic root aneurysm (HP:0002616); Mitral valve prolapse (HP:0001634); Mitral regurgitation (HP:0001653)
D27 (MALE; P35Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.4615C>T (heterozygous),Tall stature (HP:0000098); Ectopia lentis (HP:0001083); Pectus carinatum (HP:0000768); High palate (HP:0000218); Dolichocephaly (HP:0000268); Myopia (HP:0000545); Aortic root aneurysm (HP:0002616); Spontaneous pneumothorax (HP:0002108)
D36 (FEMALE; P11Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.4217A>G (heterozygous),Tall stature (HP:0000098); Arachnodactyly (HP:0001166); Joint hypermobility (HP:0001382); Ectopia lentis (HP:0001083); Aortic root aneurysm (HP:0002616); Mitral valve prolapse (HP:0001634)


In [21]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              metadata=metadata,
                                              outdir=output_directory)

We output 12 GA4GH phenopackets to the directory phenopackets


In [ ]:
# pxf validate --hpo ../hpo_data/hp.json *.json
# no errors